### Task 2.2: WA Counties with the Largest Populations

In [60]:
import pandas as pd
import geopandas

#read files into geopandas
waShape = geopandas.read_file("saep_bg10\saep_bg10.shp")
countyName = geopandas.read_file("WashingtonFIPS.dbf")

#join the shapefile and county name file together
wa_st = pd.merge(waShape, countyName, left_on = ['COUNTYFP10'], right_on = ['FIPSCounty'], how = 'left')

#create dataframe of countyname and 2017 population
cnty_pop = wa_st[['CountyName', 'POP2017']]

#additional functions to dataframe
cnty_pop = cnty_pop.groupby(['CountyName'])
cnty_pop = cnty_pop.sum()
cnty_pop = cnty_pop.sort_values(by = ['POP2017'], ascending=False)
cnty_pop.columns = ['Population (2017)']
cnty_pop.index.names = ['County Name']

#print out county dataframe of county name and population
cnty_pop.head(10)


,Population (2017)
County Name,
King,2153700.0
Pierce,859400.0
Snohomish,789400.0
Spokane,499800.0
Clark,471000.0
Thurston,276900.0
Kitsap,264300.0
Yakima,253000.0
Whatcom,216300.0
